In [1]:
# Imports 
import string
import pandas as pd
import numpy as np
import time
from scipy.stats.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
# load the data, keep only selected columns
# Read the file using super fast Pandas.read_csv
def load_required_data(path, required_columns):
    dataframe = pd.read_json(path, lines=True)
    dataframe = dataframe[required_columns]
    return dataframe, dataframe.values

In [5]:
dataframe, values = load_required_data('train_15000.json', ["asin", "reviewerID", "overall"])

In [6]:
dataframe[:10]

,asin,reviewerID,overall
0,B00ANT8OF6,A00100742Q4O8VH0YMUBZ,4
1,B00AMR1HZ8,A00100742Q4O8VH0YMUBZ,4
2,B004WGGQPQ,A00100742Q4O8VH0YMUBZ,5
3,B006C1ZSO4,A00100742Q4O8VH0YMUBZ,5
4,B008LY1B32,A00100742Q4O8VH0YMUBZ,4
5,B007KPT2N4,A00100742Q4O8VH0YMUBZ,1
6,B0080JJLBW,A00100742Q4O8VH0YMUBZ,5
7,B009P8EMCK,A00100742Q4O8VH0YMUBZ,5
8,B00I8Q77Y0,A001619027H9L9EG4UVRB,5
9,B00H0BGCJK,A001619027H9L9EG4UVRB,5


In [7]:
dataframe.shape

(15000, 3)

In [8]:
# Create user-item matrix given a matrix and mapping from original matrix columns to new matrix rows/columns
def create_user_item_matrix(data, rowMapping, columnMapping):
    # Create table of users to items by taking row and column pivots
    # Rows of user-item table are users
    rows, row_pos = np.unique(data[:, rowMapping], return_inverse=True)
    # Columns of user-item table are movies
    cols, col_pos = np.unique(data[:, columnMapping], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_table[row_pos, col_pos] = data[:, 2]
    return pivot_table, rows, cols

In [9]:
ratings_matrix, rows, cols = create_user_item_matrix(values, rowMapping=1, columnMapping=0)
print (ratings_matrix.shape)

(3139, 1842)


In [10]:
pd.DataFrame(ratings_matrix[:10], index = rows[:10], columns= cols)

,B004ALVL6W,B004ANMWPY,B004DLNC4I,B004DLPXAO,B004DM1OAQ,B004DM1ZQY,B004DPBGCO,B004DPC5Y2,B004DPCSKI,B004DPIEF6,...,B00K7WGUKA,B00KFNXUY0,B00KGCNRAM,B00KI5Q8X0,B00KMX5V8G,B00KOEHQCW,B00KQHVWWC,B00KSOQ66K,B00KWVZ750,B00L3MNCNQ
A00100742Q4O8VH0YMUBZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A001619027H9L9EG4UVRB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A002359833QJM7OQHCXWY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0037670NPLI11RBWYFA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A003841815JTX0JFLR8B1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0038872349TB5N0JHQQW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00455683H6M1GQZMPQPV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
A0047670XAAJD587LXS7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00501041JRAPWYLPQ4TE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00507662MEMHI1YMGQ15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
def get_index_map(arr):
    index_map = {}
    for el in arr:
        index_map[el] = len(index_map)
    return index_map

In [12]:
def pearson_coeff(target, others):
    return np.corrcoef(matrix[target], matrix[others])[0][1:]

In [13]:
# Get unique movies and users
unique_items = np.unique(values[:,0])
unique_users = np.unique(values[:,1])

In [14]:
unique_items.shape

(1842,)

In [15]:
unique_users.shape

(3139,)

In [16]:
# Create mapping from unique list to 
item_idx_map = get_index_map(unique_items)
user_idx_map = get_index_map(unique_users)

In [17]:
mean_ratings = np.true_divide(ratings_matrix.sum(1), (ratings_matrix!=0).sum(1))

In [53]:
K = 0.01

In [54]:
def get_users_who_rated_item(item_id):
    return np.where(ratings_matrix[:,item_idx_map[item_id]] > 0)[0]

def get_users_who_rated_item_by_index(item_idx):
    return np.where(ratings_matrix[:,item_idx] > 0)

def predict(user_id, item_id):
    item_idx = item_idx_map[item_id]
    user_idx = user_idx_map[user_id]
    other_users_idx = get_users_who_rated_item(item_id)
    if user_idx in other_users_idx:
        np.delete(other_users_idx, user_idx)
    pearson_correlation = [pearsonr(other_user, ratings_matrix[user_idx])[0] for other_user in ratings_matrix[other_users_idx]]
    mean_others = mean_ratings[other_users_idx]
    rating_others = ratings_matrix[other_users_idx,item_idx]
    prediction = mean_ratings[user_idx] + K * np.sum(pearson_correlation * (rating_others - mean_others))
    return prediction

In [47]:
np.mean(mean_ratings)

4.100578451642274

In [48]:
test_dataframe, test_values = load_required_data('test_5000.json', ["asin", "reviewerID", "overall"])

In [49]:
test_dataframe['reviewerID'].unique().shape

(2507,)

In [55]:
# Returns DF containing predictions, MAE, RMSE
def get_predictions(dataframe):
    df = dataframe.copy()
    print ("Predicting ratings..")
    start_time = time.time()
    df['Prediction'] = df.apply(lambda row: predict_row(row) , axis=1)
    MAE = round(mean_absolute_error(df['overall'], df['Prediction']), 2)
    RMSE = round(np.sqrt(mean_squared_error(df['overall'], df['Prediction'])), 2)
    print "Done Predicting in",  round((time.time() - start_time), 2), "seconds"
    return df, MAE, RMSE

i = 0
def predict_row(row):
    global i
    i+= 1
    prediction = predict(row['reviewerID'], row['asin'])
    print ("#", i, "Actual = ", row['overall'], "Predictioned = ", prediction)
    return prediction

In [56]:
df, MAE, RMSE = get_predictions(test_dataframe)

Predicting ratings..
('#', 1, 'Actual = ', 5, 'Predictioned = ', 4.124921863257795)
('#', 2, 'Actual = ', 2, 'Predictioned = ', 4.124686964959625)
('#', 3, 'Actual = ', 5, 'Predictioned = ', 5.000037299243523)
('#', 4, 'Actual = ', 5, 'Predictioned = ', 5.003835513605458)
('#', 5, 'Actual = ', 5, 'Predictioned = ', 4.249889542052348)
('#', 6, 'Actual = ', 3, 'Predictioned = ', 4.2509748591588625)
('#', 7, 'Actual = ', 5, 'Predictioned = ', 2.9999706202658962)
('#', 8, 'Actual = ', 4, 'Predictioned = ', 3.000095640387044)
('#', 9, 'Actual = ', 5, 'Predictioned = ', 4.999971712591185)
('#', 10, 'Actual = ', 5, 'Predictioned = ', 5.010910011931595)
('#', 11, 'Actual = ', 1, 'Predictioned = ', 4.500346380952835)
('#', 12, 'Actual = ', 4, 'Predictioned = ', 4.500148955941891)
('#', 13, 'Actual = ', 5, 'Predictioned = ', 4.997795987770445)
('#', 14, 'Actual = ', 5, 'Predictioned = ', 5.000064608528643)
('#', 15, 'Actual = ', 4, 'Predictioned = ', 3.9944093115353914)
('#', 16, 'Actual = ', 1,

('#', 134, 'Actual = ', 4, 'Predictioned = ', 3.668992946266665)
('#', 135, 'Actual = ', 5, 'Predictioned = ', 4.9973322646302085)
('#', 136, 'Actual = ', 5, 'Predictioned = ', 4.999992069589031)
('#', 137, 'Actual = ', 5, 'Predictioned = ', 5.000064591409791)
('#', 138, 'Actual = ', 3, 'Predictioned = ', 4.999902811062608)
('#', 139, 'Actual = ', 4, 'Predictioned = ', 3.834348229634795)
('#', 140, 'Actual = ', 4, 'Predictioned = ', 3.8379213075634473)
('#', 141, 'Actual = ', 5, 'Predictioned = ', 2.6659911379344217)
('#', 142, 'Actual = ', 1, 'Predictioned = ', 2.6666302576609873)
('#', 143, 'Actual = ', 5, 'Predictioned = ', 5.000487747456724)
('#', 144, 'Actual = ', 4, 'Predictioned = ', 5.00168143506229)
('#', 145, 'Actual = ', 5, 'Predictioned = ', 3.9998998121143474)
('#', 146, 'Actual = ', 4, 'Predictioned = ', 4.000180002495113)
('#', 147, 'Actual = ', 5, 'Predictioned = ', 4.4300124803644865)
('#', 148, 'Actual = ', 5, 'Predictioned = ', 4.427421191498596)
('#', 149, 'Actual =

('#', 260, 'Actual = ', 5, 'Predictioned = ', 4.665739938595892)
('#', 261, 'Actual = ', 5, 'Predictioned = ', 2.9999577208694856)
('#', 262, 'Actual = ', 5, 'Predictioned = ', 3.0000063232756817)
('#', 263, 'Actual = ', 5, 'Predictioned = ', 4.000158056944796)
('#', 264, 'Actual = ', 5, 'Predictioned = ', 3.9999866663269192)
('#', 265, 'Actual = ', 1, 'Predictioned = ', 2.8000591936861747)
('#', 266, 'Actual = ', 1, 'Predictioned = ', 2.797647041625755)
('#', 267, 'Actual = ', 5, 'Predictioned = ', 4.402256553124703)
('#', 268, 'Actual = ', 4, 'Predictioned = ', 4.399592444115139)
('#', 269, 'Actual = ', 4, 'Predictioned = ', 4.235209249280635)
('#', 270, 'Actual = ', 4, 'Predictioned = ', 3.9985062576107193)
('#', 271, 'Actual = ', 3, 'Predictioned = ', 3.996356310304584)
('#', 272, 'Actual = ', 5, 'Predictioned = ', 4.664319822762758)
('#', 273, 'Actual = ', 5, 'Predictioned = ', 4.666702612950297)
('#', 274, 'Actual = ', 1, 'Predictioned = ', 3.4999693841928723)
('#', 275, 'Actual 

('#', 394, 'Actual = ', 5, 'Predictioned = ', 5.00188902322278)
('#', 395, 'Actual = ', 1, 'Predictioned = ', 4.999866156265553)
('#', 396, 'Actual = ', 2, 'Predictioned = ', 2.9998941280191294)
('#', 397, 'Actual = ', 2, 'Predictioned = ', 2.993450957366279)
('#', 398, 'Actual = ', 3, 'Predictioned = ', 4.142858447289942)
('#', 399, 'Actual = ', 3, 'Predictioned = ', 4.142484263757673)
('#', 400, 'Actual = ', 5, 'Predictioned = ', 5.000076997624136)
('#', 401, 'Actual = ', 1, 'Predictioned = ', 5.000184452976768)
('#', 402, 'Actual = ', 5, 'Predictioned = ', 2.999954113017638)
('#', 403, 'Actual = ', 5, 'Predictioned = ', 2.9999760826235287)
('#', 404, 'Actual = ', 4, 'Predictioned = ', 4.000000803484395)
('#', 405, 'Actual = ', 4, 'Predictioned = ', 3.9997065124032187)
('#', 406, 'Actual = ', 5, 'Predictioned = ', 4.502658408621288)
('#', 407, 'Actual = ', 5, 'Predictioned = ', 4.500125322866758)
('#', 408, 'Actual = ', 5, 'Predictioned = ', 4.333930862320118)
('#', 409, 'Actual = ',

('#', 525, 'Actual = ', 5, 'Predictioned = ', 4.670737140517492)
('#', 526, 'Actual = ', 4, 'Predictioned = ', 4.999987487909932)
('#', 527, 'Actual = ', 5, 'Predictioned = ', 5.000425933462035)
('#', 528, 'Actual = ', 2, 'Predictioned = ', 4.751139418828994)
('#', 529, 'Actual = ', 5, 'Predictioned = ', 4.749852005180364)
('#', 530, 'Actual = ', 5, 'Predictioned = ', 4.999993149195072)
('#', 531, 'Actual = ', 5, 'Predictioned = ', 5.007719548741325)
('#', 532, 'Actual = ', 5, 'Predictioned = ', 4.666528174230542)
('#', 533, 'Actual = ', 5, 'Predictioned = ', 4.667057862755639)
('#', 534, 'Actual = ', 5, 'Predictioned = ', 5.0024748751714005)
('#', 535, 'Actual = ', 5, 'Predictioned = ', 5.000092429717184)
('#', 536, 'Actual = ', 5, 'Predictioned = ', 2.99996542534878)
('#', 537, 'Actual = ', 5, 'Predictioned = ', 2.996019825343752)
('#', 538, 'Actual = ', 3, 'Predictioned = ', 3.24067573652852)
('#', 539, 'Actual = ', 3, 'Predictioned = ', 3.243975922663649)
('#', 540, 'Actual = ', 1,

...

('#', 4418, 'Actual = ', 5, 'Predictioned = ', 3.7592441133207894)
('#', 4419, 'Actual = ', 5, 'Predictioned = ', 2.9999804925286524)
('#', 4420, 'Actual = ', 3, 'Predictioned = ', 2.9998261608097154)
('#', 4421, 'Actual = ', 4, 'Predictioned = ', 3.5000074089954585)
('#', 4422, 'Actual = ', 4, 'Predictioned = ', 3.4983918898935586)
('#', 4423, 'Actual = ', 3, 'Predictioned = ', 4.333384571390016)
('#', 4424, 'Actual = ', 4, 'Predictioned = ', 4.33401001705947)
('#', 4425, 'Actual = ', 5, 'Predictioned = ', 4.7483165869105)
('#', 4426, 'Actual = ', 5, 'Predictioned = ', 4.750388745474604)
('#', 4427, 'Actual = ', 5, 'Predictioned = ', 4.669066008119954)
('#', 4428, 'Actual = ', 5, 'Predictioned = ', 4.669621227691259)
('#', 4429, 'Actual = ', 4, 'Predictioned = ', 2.3333243072140775)
('#', 4430, 'Actual = ', 4, 'Predictioned = ', 2.3362904661188946)
('#', 4431, 'Actual = ', 2, 'Predictioned = ', 4.333355617096315)
('#', 4432, 'Actual = ', 5, 'Predictioned = ', 4.331427090321876)
('#', 

('#', 4543, 'Actual = ', 5, 'Predictioned = ', 0.9994220428848906)
('#', 4544, 'Actual = ', 5, 'Predictioned = ', 4.666946640734213)
('#', 4545, 'Actual = ', 5, 'Predictioned = ', 4.666623452664359)
('#', 4546, 'Actual = ', 5, 'Predictioned = ', 5.000807901283511)
('#', 4547, 'Actual = ', 4, 'Predictioned = ', 5.0000928142514764)
('#', 4548, 'Actual = ', 5, 'Predictioned = ', 4.999898126276997)
('#', 4549, 'Actual = ', 5, 'Predictioned = ', 5.000003471024408)
('#', 4550, 'Actual = ', 2, 'Predictioned = ', 1.0000384660421109)
('#', 4551, 'Actual = ', 5, 'Predictioned = ', 0.9999927800663747)
('#', 4552, 'Actual = ', 3, 'Predictioned = ', 4.223621650841449)
('#', 4553, 'Actual = ', 5, 'Predictioned = ', 4.222179600291333)
('#', 4554, 'Actual = ', 4, 'Predictioned = ', 4.666715383947669)
('#', 4555, 'Actual = ', 5, 'Predictioned = ', 4.663999499632839)
('#', 4556, 'Actual = ', 4, 'Predictioned = ', 3.342117686640832)
('#', 4557, 'Actual = ', 4, 'Predictioned = ', 3.3332770504957034)
('#',

('#', 4693, 'Actual = ', 5, 'Predictioned = ', 5.007005902836662)
('#', 4694, 'Actual = ', 5, 'Predictioned = ', 5.000017155726595)
('#', 4695, 'Actual = ', 5, 'Predictioned = ', 4.750299790521839)
('#', 4696, 'Actual = ', 4, 'Predictioned = ', 4.74991183521286)
('#', 4697, 'Actual = ', 4, 'Predictioned = ', 4.000217184366865)
('#', 4698, 'Actual = ', 5, 'Predictioned = ', 3.9996442591269115)
('#', 4699, 'Actual = ', 4, 'Predictioned = ', 2.5999531179598483)
('#', 4700, 'Actual = ', 1, 'Predictioned = ', 2.6001187258029907)
('#', 4701, 'Actual = ', 5, 'Predictioned = ', 4.876720269758144)
('#', 4702, 'Actual = ', 5, 'Predictioned = ', 4.874984724304765)
('#', 4703, 'Actual = ', 4, 'Predictioned = ', 4.199712489810283)
('#', 4704, 'Actual = ', 1, 'Predictioned = ', 4.200091060088827)
('#', 4705, 'Actual = ', 3, 'Predictioned = ', 2.6000158944514826)
('#', 4706, 'Actual = ', 5, 'Predictioned = ', 2.5999792206279038)
('#', 4707, 'Actual = ', 4, 'Predictioned = ', 2.9990374863101863)
('#',

('#', 4830, 'Actual = ', 4, 'Predictioned = ', 2.9855756160248608)
('#', 4831, 'Actual = ', 1, 'Predictioned = ', 3.000015991000525)
('#', 4832, 'Actual = ', 5, 'Predictioned = ', 4.667093757003722)
('#', 4833, 'Actual = ', 5, 'Predictioned = ', 4.660858706103326)
('#', 4834, 'Actual = ', 3, 'Predictioned = ', 5.000786513882428)
('#', 4835, 'Actual = ', 5, 'Predictioned = ', 4.999994240400465)
('#', 4836, 'Actual = ', 4, 'Predictioned = ', 4.666632724252879)
('#', 4837, 'Actual = ', 4, 'Predictioned = ', 4.66363366932745)
('#', 4838, 'Actual = ', 5, 'Predictioned = ', 3.7499887559049845)
('#', 4839, 'Actual = ', 5, 'Predictioned = ', 3.750568829189336)
('#', 4840, 'Actual = ', 5, 'Predictioned = ', 4.009576906229878)
('#', 4841, 'Actual = ', 5, 'Predictioned = ', 3.997931128873885)
('#', 4842, 'Actual = ', 5, 'Predictioned = ', 4.499979808363181)
('#', 4843, 'Actual = ', 5, 'Predictioned = ', 4.499990025159789)
('#', 4844, 'Actual = ', 5, 'Predictioned = ', 3.600543629916636)
('#', 484

('#', 4956, 'Actual = ', 2, 'Predictioned = ', 5.0031605051812456)
('#', 4957, 'Actual = ', 5, 'Predictioned = ', 4.995886774624187)
('#', 4958, 'Actual = ', 2, 'Predictioned = ', 5.001644017932928)
('#', 4959, 'Actual = ', 5, 'Predictioned = ', 4.402295137255029)
('#', 4960, 'Actual = ', 5, 'Predictioned = ', 4.399957187498607)
('#', 4961, 'Actual = ', 4, 'Predictioned = ', 3.5043433441074217)
('#', 4962, 'Actual = ', 1, 'Predictioned = ', 3.5002210468795623)
('#', 4963, 'Actual = ', 5, 'Predictioned = ', 3.5442991096514307)
('#', 4964, 'Actual = ', 3, 'Predictioned = ', 3.546118374937345)
('#', 4965, 'Actual = ', 3, 'Predictioned = ', 4.073662984750278)
('#', 4966, 'Actual = ', 5, 'Predictioned = ', 4.068975636588188)
('#', 4967, 'Actual = ', 4, 'Predictioned = ', 4.798649888132492)
('#', 4968, 'Actual = ', 5, 'Predictioned = ', 4.7995821523313875)
('#', 4969, 'Actual = ', 5, 'Predictioned = ', 4.663309540897259)
('#', 4970, 'Actual = ', 5, 'Predictioned = ', 4.670877930732574)
('#',

In [57]:
print "MAE=", MAE
print "RMSE=", RMSE

MAE= 0.93
RMSE= 1.33


In [ ]:
sparse_ratings = csr_matrix(ratings_matrix, dtype='float')
model_knn = NearestNeighbors(metric = 'cosine', n_neighbors = 10)
model_knn.fit(sparse_ratings)
distances,indices = model_knn.kneighbors(ratings_matrix[0].reshape(1, -1), n_neighbors=6)